Generate synthetic libraries using NK model

In [1]:
import torch
from torch import distributions as dist
import itertools

import operator
import pickle
import importlib
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('white')
sns.set_context('paper')
# Plot adjustments:
plt.rcParams.update({'ytick.labelsize': 15})
plt.rcParams.update({'xtick.labelsize': 15})
plt.rcParams.update({'axes.labelsize': 35})
plt.rcParams.update({'legend.fontsize': 30})
plt.rcParams.update({'axes.titlesize': 16})

from gptorch import kernels, models
import bases, helpers, opt

In [2]:
class NK_Model(object):
    """ A Kauffman NK Model, with tuneable ruggedness K. 
    
    N parts the amino acids in the primary sequence
    A states are the 20 possible amino acids (A takes on values from 0 to 19)
    K measures the average number of other sites in the primary chain whose 
    amino acids bear on the functional contribution of the amino acid at a 
    given site to overall function. """
        
    # I would use a gaussian distribution for the random effects
    def __init__(self, N=4, K=1, A=20):
        """ Initialize a NK Model."""
        self.N = N
        self.K = K
        self.A = A
        self.graph = self.create_graph()
            
    def create_graph(self):
        '''        
        Returns a dictionary of N tuples of size K + 1 with their corresponding
        fitnesses drawn randomly from Gaussian distribution. Each tuple consists 
        of a specific amino acid position that is influenced by K other positions
        included in the tuple. There are A total possible amino acids at each position.
        '''
        graph = {}
        
        poss = [i for i in range(self.A)]
        nbds = [tuple(i) for i in itertools.product(poss, repeat = self.K + 1)] # all possible neighborhoods

        for i in nbds:
            graph[i] = np.random.normal(0, 1, 1)[0] # draw from Gaussian dist
            
        return graph
    
    def seq_nbds(self, seq):
        '''
        Returns a list of all contacts/neighborhoods in a given sequence.
        Each tuple consists of a specific amino acid position that is influenced 
        by K other positions included in the tuple. For each contact/neighborhood,
        we take the K positions before specific site to be in the tuple.
        '''
        if len(seq) != self.N:
            raise Exception("Wrong sequence length.")
            
        seq_nbrs = []
        
        for i in range(self.N):
            lst = [] # temp lst
            ind = i
            for k in range(self.K + 1): # take K positions before specific site
                lst.append(seq[ind])
                ind -= 1
            seq_nbrs.append(tuple(lst))
            
        return seq_nbrs
    
    def eval_fitness(self, seq):
        '''
        Returns value of fitness of an entire amino acid sequence.
        '''
        if len(seq) != self.N:
            raise Exception("Wrong sequence length.")
        
        nbrs = self.seq_nbds(seq)
        fits = [self.graph[n] for n in nbrs]
        
        return sum(fits) / len(fits) # take average

    def all_seqs(self):
        '''
        Generates a list of all possible sequences of length N 
        where each position could take on A different 
        values for each of the amino acids.
        '''
        poss = [i for i in range(self.A)] # all values of A
        seqs = [tuple(i) for i in itertools.product(poss, repeat = self.N)] # stores all possible seqs
        return seqs
        
    def enumerate_space(self):
        '''
        Exhaustively enumerate the fitness of each point in the model space.
        
        Returns dictionary of seq with corresponding fitness
        '''
        model_space = {}
        seqs = self.all_seqs() # generate all seqs in landscape
        
        for seq in seqs: # evaluate fitness of each seq
            model_space[seq] = self.eval_fitness(seq)
            
        return model_space

In [3]:
nk = NK_Model()

In [4]:
space = nk.enumerate_space()

In [5]:
space

{(0, 0, 0, 0): -1.1843565857113936,
 (0, 0, 0, 1): -0.36635160815283557,
 (0, 0, 0, 2): -0.85689034762126359,
 (0, 0, 0, 3): 0.032473673793078861,
 (0, 0, 0, 4): -0.80802935675039311,
 (0, 0, 0, 5): -1.0715695054224579,
 (0, 0, 0, 6): -0.29417056654075696,
 (0, 0, 0, 7): -0.91275050980677386,
 (0, 0, 0, 8): -0.33600127631731791,
 (0, 0, 0, 9): -0.78014248871973857,
 (0, 0, 0, 10): -1.1505019555853224,
 (0, 0, 0, 11): -1.3537271507333051,
 (0, 0, 0, 12): -0.61530808793741398,
 (0, 0, 0, 13): -0.35439312916269555,
 (0, 0, 0, 14): -0.28125581988726583,
 (0, 0, 0, 15): -0.64794640693775574,
 (0, 0, 0, 16): -0.53796970805039412,
 (0, 0, 0, 17): -0.76988569765674897,
 (0, 0, 0, 18): -0.51218171853233074,
 (0, 0, 0, 19): -0.27088339627276226,
 (0, 0, 1, 0): -0.36635160815283557,
 (0, 0, 1, 1): 0.15523577366663763,
 (0, 0, 1, 2): -0.21822515856798352,
 (0, 0, 1, 3): 0.40985360112773533,
 (0, 0, 1, 4): 0.0069812529776553597,
 (0, 0, 1, 5): -0.66671999764892509,
 (0, 0, 1, 6): 0.2558508613549032